# Building Connect Four
To finish this module, you will create a minimax function for a game of [Connect Four](https://codehs.com/share/id/connect-four-LuXrGD/run).

Connect Four is similar to Tic Tac Toe - to win, you need to connect four of your pieces in a row, in a column, or diagonal. If you’re unfamiliar with the rules, you can check them out here.

Your finished product should look something like this game here. This version of the game uses minimax with alpha beta pruning and depth to analyze the board and make a move. Play a couple of times to see if you can beat it!

# Connect Four
For this assignment, you will implement a minimax algorithm from scratch that will allow users to play an AI in Connect Four.

All of the code has already been written to get the game to work - all you need to implement is a working minimax function.

There is a lot of code already included in this program - most of it you won’t have to do anything with at all, so don’t worry. There are also comments that describe what every function does.

Complete the `minimax()` function on line **356**. To implement minimax, you will need to rely on the following functions:

1. `check_win(player`): This function returns true if a player has won the game. This should be used in the base case.

2. `check_moves_left()`: This function returns true if there are moves left in the game. If this is false, then the game has ended in a tie. This function should be used in your base case.

3. `check_curr_score()`: This function returns the current score of the game board. In Tic Tac Toe, we returned 0 when depth of 0 was reached, but this doesn’t work with Connect Four. Because there are so many more potential moves, certain board set ups are more advantageous than others. For example, if a player has three in a row and an available space to place a fourth, that is more valuable than if three are in a row but one is blocked. This function checks for all available moves and evaluates the game board with a score:

Available Black Moves - Available Red Moves.

This function should be used in the base case for when the depth reaches zero. The value of `check_curr_score()` should be returned to minimax.

4. `check_column_depth(col)`: This function returns the correct row to place a player. In Connect Four, players only need to choose a column. The row that the piece is placed at is determined by how many pieces are already in that column. This function takes a column and returns the row that a player can be placed at. When iterating through all potential moves, you only need to iterate through each column, then call `check_col_depth` to determine which row that piece should be placed at.

5. `place_player(player, row, col)`: This function places a player on the board at the row, col that is specified. This should be called before the next call to minimax so that the new player piece is added before the next evaluation.

In [ ]:
from browser import timer
import random

#Creates a game board and adds it to the canvas
canvas_board = Rectangle(400,300)
canvas_board.set_position(0, get_height()/5)
canvas_board.set_color(Color.blue)
player_buffer = 5
player_radius = ((canvas_board.height - (player_buffer * 7)) / 6) / 2
#Global Variable
board = []
player = "red"

#Creates a circle that is placed on the canvas at the specified x, y coordinate
def make_player(x, y, color):
    player = Circle(player_radius)
    player.set_position(x,y)
    player.set_color(color)
    add(player)

#Adds numbers to the top of each column
def add_col_nums():
    starting_x = player_radius*2
    for num in range(7):
        number = Text(num)
        number.set_position(starting_x - player_buffer, get_height()/ 5 - player_buffer)
        add(number)
        starting_x += player_radius*2 + player_buffer

#creates the initial game board for connect four
def make_board():
    for i in range(6):
        board.append(["-","-","-","-","-","-","-"])

#Iterates through game board and places circles on canvas based on the current board
def add_curr_players():
    starting_x = player_radius*2 + player_buffer
    starting_y = get_height()/5 + player_buffer + player_radius
    for row in range(len(board)):
        for col in range(len(board[0])):
            if board[row][col] == "-":
                make_player(starting_x, starting_y, Color.white)
            elif board[row][col] == "red":
                make_player(starting_x, starting_y,Color.red)
            else:
                make_player(starting_x, starting_y,Color.black)
            starting_x += (player_radius* 2) + player_buffer
        starting_x = player_radius*2 + player_buffer
        starting_y += (player_radius * 2) + player_buffer

#returns true if a row,col move is a valid move on the current board
def is_valid_move(col):
    if(col >= len(board[0])) or col < 0:
        return False
    return check_column_depth(col) != - 1

#checks to see which row a player should be placed at
#players should be placed at the lowest possible row
def check_column_depth(col):
    for i in range(6):
        if(board[i][col]) != "-":
            return i - 1
    return 5

#places a player at the specified row and col
def place_player(player, row, col):
    board[row][col] = player

#Gets move from player and places that move on the board. If player is red, ask for their move
#if player is black, call minimax
def take_turn(player):
    print(player, "'s Turn")
    if player == "red":
        #try statement makes sure input is only numerical values.
        try:
            col = int(input("Enter a col "))
        except:
            col = -1
        while(not is_valid_move(col)):
            print("Please enter a valid move")
            try:
                #try statement makes sure input is only numerical values.
                col = int(input("Enter a col "))
            except:
                col = -1
            if col == "":
                return
        row = check_column_depth(col)
    else:
        #returns the optimal row, col for a given move
        score, row, col = minimax("black", 4, -100000, 100000)
    place_player(player, row, col)
    add_curr_players()

#returns true if player has won horizontally
def check_row_winner(player):
    streak = 0
    for row in range(len(board)):
        for col in range(len(board[row]) - 1):
            if board[row][col] == player and board[row][col] == board[row][col + 1]:
                streak += 1 
            else:
                streak = 0
            if streak == 3:
                return True
        streak = 0
    return False

#returns true if player has won vertically 
def check_col_winner(player):
    streak = 0
    for row in range(7):
        for col in range(5):
            if board[col][row] == player and board[col][row] == board[col + 1][row]:
                streak += 1 
            else:
                streak = 0
            if streak == 3:
                return True
        streak = 0
    return False

#returns true if player has won diagonally
#iterates through all values on board - the try/except statements catch errors
def check_diag_winner(player):
    for row in range(len(board)):
        for col in range(len(board[0])):
            try:
                if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == player and board[row + 3][col + 3] == player:
                  return True
                if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == player and board[row + 3][col-3] == player and col - 3 >= 0:
                    return True
            except IndexError:
                try:
                  if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == player and board[row + 3][col-3] == player and col - 3 >= 0:
                    return True
                except IndexError:
                    next
                next
    return False

#returns true if player has won in any way
def check_win(player):
    return check_diag_winner(player) or check_col_winner(player) or check_row_winner(player)

#returns true if there are moves left in the game
def check_moves_left():
    for list in board:
        if "-" in list:
            return True
    return False

#prints result after the game is over
def check_results():
    if check_win("black"):
        
        print("Black Wins!")
    elif check_win("red"):
        
        print("Red Wins!")
    else:
        print("It's a Tie!")


#########The following functions are used to help compute the current score of the game.###############
#You do NOT need to know how these functions work. Each function is checking to see if 
#a player has three in a row, two in a row, and one in a row. Each of those states is valuable
#for a player, and goes into the optimal move."""

def check_three_row(player):
    streak = 0
    total_threes = 0
    for row in range(len(board)):
        for col in range(len(board[row]) - 1):
            if board[row][col] == player and board[row][col] == board[row][col + 1]:
                streak += 1 
            else:
                streak = 0
            try:
                if (streak == 2 and board[row][col + 1] == "-") or (streak == 2 and board[row][col - streak - 1] == "-"):
                    total_threes += 1
            except:
                next
    return total_threes

def check_three_col(player):
    streak = 0
    total_threes = 0

    for row in range(7):
        for col in range(5):
            if board[col][row] == player and board[col][row] == board[col + 1][row]:
                streak += 1 
            else:
                streak = 0
            try:
                if (streak == 2 and board[col + 1][row] == "-") or (streak == 2 and board[col - streak - 1][row] == "-"):
                    total_threes += 1
            except: 
                next
    return total_threes

def check_three_diag(player):
    total_threes = 0
    for row in range(len(board)):
        for col in range(len(board[0])):
            try:
                if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == player and board[row + 3][col + 3] == "-":
                  total_threes += 1
                if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == player and board[row + 3][col-3] == "-":
                    total_threes += 1
                if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == player and board[row - 1][col + 1] == "-":
                    total_threes += 1
                if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == player and board[row -1][col -1] == "-":
                    total_threes += 1
            except IndexError:
                try:
                    if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == player and board[row + 3][col-3] == "-":
                        total_threes += 1
                except:
                    try:
                        if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == player and board[row - 1][col + 1] == "-":
                            total_threes += 1
                    except:
                        try:
                            if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == player and board[row -1][col -1] == "-":
                                total_threes += 1
                        except:
                            next
                next
    return total_threes

def check_two_row(player):
    streak = 0
    total_twos = 0
    for row in range(len(board)):
        for col in range(len(board[row]) - 1):
            if board[row][col] == player and board[row][col] == board[row][col + 1]:
                streak += 1 
            else:
                streak = 0
            try:
                if (streak == 1 and board[row][col + 1] == "-") or (streak == 1 and board[row][col - streak - 1] == "-"):
                    total_twos += 1
            except:
                next
    return total_twos

def check_two_col(player):
    streak = 0
    total_two = 0

    for row in range(7):
        for col in range(5):
            if board[col][row] == player and board[col][row] == board[col + 1][row]:
                streak += 1 
            else:
                streak = 0
            try:
                if (streak == 1 and board[col + 1][row] == "-") or (streak == 1 and board[col - streak - 1][row] == "-"):
                    total_two += 1
            except: 
                next
    return total_two

def check_two_diag(player):
    total_two = 0
    for row in range(len(board)):
        for col in range(len(board[0])):
            try:
                if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == "-":
                  total_two += 1
                if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == "-":
                    total_two += 1
                if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == "-":
                    total_two += 1
                if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == "-":
                    total_two += 1
            except IndexError:
                try:
                    if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == "-":
                        total_two += 1
                except:
                    try:
                        if board[row][col] == player and board[row + 1][col-1] == player and board[row + 2][col-2] == "-":
                            total_two += 1
                    except:
                        try:
                             if board[row][col] == player and board[row + 1][col + 1] == player and board[row + 2][col+2] == "-":
                                total_two += 1
                        except:
                            next
                next
    return total_two

def check_one_row(player):
    total_ones = 0
    for row in range(len(board)):
        for col in range(len(board[row]) - 1):
            try:
                if (board[row][col] == player and board[row][col + 1] == "-" and board[row][col- 1] == "-"):
                    total_ones += 1
            except:
                next
    return total_ones

def check_one_col(player):
    total_ones = 0
    for row in range(7):
        for col in range(5):
            try:
                if (board[col][row] == player and board[col + 1][row] == "-" and board[col - 1][row] == "-"):
                    total_ones += 1
            except: 
                next
    return total_ones

def check_one_diag(player):
    total_one = 0
    for row in range(len(board)):
        for col in range(len(board[0])):
            try:
                if board[row][col] == player and board[row + 1][col + 1] == "-" and board[row -1][col-1] == "-":
                  total_one += 1
                
                if board[row][col] == player and board[row + 1][col-1] == player and board[row - 1][col + 1] == "-":
                    total_one += 1
            except IndexError:
                # try:
                #   if board[row][col] == player and board[row - 1][col-1] == player and board[row - 2][col-2] == player and board[row - 3][col-3] == player:
                #     return True
                # except IndexError:
                #     pass
                next
    return total_one

def check_three_total(player):
    return (check_three_col(player) + check_three_row(player) + check_three_diag(player)) * 50

def check_two_total(player):
    return (check_two_col(player) + check_two_row(player) + check_two_diag(player)) * 20
    
def check_one_total(player):
    return (check_one_col(player) + check_one_row(player) + check_one_diag(player)) * 7


###############End of functions used to compute the current score.######################

#returns the current score of the game board. Compares the number of open black winning move
#to the number of red winning moves. This function should be used in minimax!
def check_curr_score():
    return check_three_total("black") + check_two_total("black") + check_one_total("black") - check_two_total("red") - check_three_total("red") - check_one_total("red")

#Implement minimax with depth and alpha beta pruning.
#returns a score, row and col value.
def minimax(player, depth, alpha, beta):
    return

#starts the game!
def play_game():
    global player
    add_curr_players()
    take_turn(player)
    if player == "red":
        player = "black"
    else:
        player = "red"
    if not check_win("black") and not check_win("red") and check_moves_left():
        print("Next Move...")
        #timer is used to delay the start of the next move. This is necessary for
        #the graphics to work.
        timer.set_timeout(play_game, 100)
    else:
        check_results()
        add_curr_players()

add(canvas_board)
make_board()
add_col_nums()
add_curr_players()
#timer is used to delay the start of the next move. This is necessary for
#the graphics to work.
timer.set_timeout(play_game, 100)


# Challenge: Create Your Own Game
As an optional activity, consider creating an AI game of your own!

You can try creating Connect Four from scratch, or try to create other games that make use of minimax like:

- Chess
- Go
- Backgammon

If you want to build a game with graphics, make sure to use the Python graphics (Brython) type, otherwise, you can use the Python 3 type.